# 프로젝트 설계

### 1. 주제

 - 행정동별 각 시간대 생활인구 수 추이를 파악하고, 이에 기반하여 시간대별 집중 방역 지역을 선정

### 2. Dataset

 - 서울시 열린 데이터 광장, '행정동별 서울 생활인구 ( 2017 ~ 2019년 3/4월 + 2020년 3월)'
 - 코로나 데이터

### 3. Plan

- 2017 ~ 2019년 3월/4월 생활인구 데이터를 비교하여 3월 대비 4월 생활인구 수가 증가하는 지역을 선정 ( + 인구 특성 파악 )
- 행정동별 생활인구 증가율이 가장 높은 시간대를 선정하여 방역 자원 집중
- 구별 확진자 증가 추이가 급격한 지역을 찾아 구의 인구 특성 파악

In [1]:
import pandas as pd
import numpy as np

## 행정동 코드 전처리

In [2]:
loc_code = pd.read_csv('dataset/LOCAL_PEOPLE_DONG_201904/code.csv', encoding='euc-kr')

In [3]:
loc_code.head()

,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일
0,47900,40035,0,경상북도,예천군,지보면 마산리,NaN,19880423,99991231
1,47930,38022,0,경상북도,울진군,후포면 삼율리,NaN,19880423,99991231
2,47930,38023,0,경상북도,울진군,후포면 금음리,NaN,19880423,99991231
3,47940,25000,0,경상북도,울릉군,울릉읍,NaN,19880423,99991231
4,47940,25024,0,경상북도,울릉군,울릉읍 도동리,NaN,19880423,99991231


In [4]:
loc_code['시군구코드'] = loc_code['시군구코드'].astype(str)
loc_code['행정동코드'] = loc_code['행정동코드'].astype(str)
loc_code['행정동코드_'] = loc_code['시군구코드'] + loc_code['행정동코드']

In [5]:
loc_code = loc_code[loc_code['시도명'] == '서울특별시']
loc_code = loc_code[loc_code['행정동명'].isna() == False]
loc_code = loc_code[['행정동코드_', '시도명', '시군구명', '행정동명']]
loc_code['행정동명_'] = loc_code['시도명'] + ' ' + loc_code['시군구명'] + ' ' + loc_code['행정동명']
loc_code = loc_code[['시군구명', '행정동코드_', '행정동명_']]

In [6]:
loc_code = loc_code.drop_duplicates()

In [7]:
loc_code.isna().sum()

시군구명      0
행정동코드_    0
행정동명_     0
dtype: int64

In [8]:
loc_code[loc_code['시군구명']=='중구']

,시군구명,행정동코드_,행정동명_
2182,중구,11140520,서울특별시 중구 소공동
2194,중구,11140540,서울특별시 중구 회현동
2202,중구,11140550,서울특별시 중구 명동
2225,중구,11140570,서울특별시 중구 필동
2237,중구,11140580,서울특별시 중구 장충동
2240,중구,11140590,서울특별시 중구 광희동
2250,중구,11140605,서울특별시 중구 을지로동
2260,중구,11140650,서울특별시 중구 신당제5동
2261,중구,11140670,서울특별시 중구 황학동
2262,중구,11140680,서울특별시 중구 중림동


## 2019년 서울시 중구 소공동 3월/4월 생활인구 수 비교

In [9]:
ppl_1903 = pd.read_csv('dataset/LOCAL_PEOPLE_DONG_201903/LOCAL_PEOPLE_DONG_201903.csv')

# Fixing the error in index

col_names = ppl_1903.columns
ppl_1903 = ppl_1903.reset_index()
ppl_1903 = ppl_1903.iloc[:,:len(col_names)]
ppl_1903.columns = col_names
ppl_1903_ = ppl_1903.copy().round().astype('int64')

In [10]:
ppl_1903_.head()

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20190301,0,11170700,13407,151,103,291,476,755,664,...,680,617,688,509,549,359,491,524,415,1011
1,20190301,0,11650651,46327,1867,664,852,1181,1692,1690,...,1743,1795,2137,1983,1839,1512,1749,1831,1591,3111
2,20190301,0,11680730,20998,591,415,674,531,679,679,...,808,662,779,690,845,870,1014,1116,752,1824
3,20190301,0,11680670,11948,433,364,455,425,274,248,...,267,212,408,582,620,515,523,518,408,629
4,20190301,0,11680740,16430,572,366,679,453,440,471,...,462,544,761,693,881,629,710,694,460,1224


In [11]:
# 결측치 확인

ppl_1903_.isna().sum()

기준일ID              0
시간대구분              0
행정동코드              0
총생활인구수             0
남자0세부터9세생활인구수      0
남자10세부터14세생활인구수    0
남자15세부터19세생활인구수    0
남자20세부터24세생활인구수    0
남자25세부터29세생활인구수    0
남자30세부터34세생활인구수    0
남자35세부터39세생활인구수    0
남자40세부터44세생활인구수    0
남자45세부터49세생활인구수    0
남자50세부터54세생활인구수    0
남자55세부터59세생활인구수    0
남자60세부터64세생활인구수    0
남자65세부터69세생활인구수    0
남자70세이상생활인구수       0
여자0세부터9세생활인구수      0
여자10세부터14세생활인구수    0
여자15세부터19세생활인구수    0
여자20세부터24세생활인구수    0
여자25세부터29세생활인구수    0
여자30세부터34세생활인구수    0
여자35세부터39세생활인구수    0
여자40세부터44세생활인구수    0
여자45세부터49세생활인구수    0
여자50세부터54세생활인구수    0
여자55세부터59세생활인구수    0
여자60세부터64세생활인구수    0
여자65세부터69세생활인구수    0
여자70세이상생활인구수       0
dtype: int64

In [12]:
ppl_1904 = pd.read_csv('dataset/LOCAL_PEOPLE_DONG_201904/LOCAL_PEOPLE_DONG_201904.csv')

# Fixing the error in index

col_names = ppl_1904.columns
ppl_1904 = ppl_1904.reset_index()
ppl_1904 = ppl_1904.drop('여자70세이상생활인구수', axis = 1)
ppl_1904.columns = col_names

ppl_1904_ = ppl_1904.copy().round().astype('int64')

In [13]:
ppl_1904_.head()

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20190401,0,11305555,28876,936,518,600,709,920,860,...,935,851,1044,974,1224,1188,1292,1122,928,2908
1,20190401,0,11320511,24891,679,408,586,633,776,720,...,858,998,955,892,1211,1041,1114,1238,730,2173
2,20190401,0,11500550,17481,733,333,379,370,503,756,...,581,783,841,771,691,562,762,714,602,1178
3,20190401,0,11320513,14025,547,305,406,361,423,384,...,406,364,568,549,635,645,621,481,402,1443
4,20190401,0,11620775,22754,521,308,617,693,785,854,...,791,803,778,751,903,906,1138,840,700,1905


In [14]:
# 결측치 확인

ppl_1904_.isna().sum()

기준일ID              0
시간대구분              0
행정동코드              0
총생활인구수             0
남자0세부터9세생활인구수      0
남자10세부터14세생활인구수    0
남자15세부터19세생활인구수    0
남자20세부터24세생활인구수    0
남자25세부터29세생활인구수    0
남자30세부터34세생활인구수    0
남자35세부터39세생활인구수    0
남자40세부터44세생활인구수    0
남자45세부터49세생활인구수    0
남자50세부터54세생활인구수    0
남자55세부터59세생활인구수    0
남자60세부터64세생활인구수    0
남자65세부터69세생활인구수    0
남자70세이상생활인구수       0
여자0세부터9세생활인구수      0
여자10세부터14세생활인구수    0
여자15세부터19세생활인구수    0
여자20세부터24세생활인구수    0
여자25세부터29세생활인구수    0
여자30세부터34세생활인구수    0
여자35세부터39세생활인구수    0
여자40세부터44세생활인구수    0
여자45세부터49세생활인구수    0
여자50세부터54세생활인구수    0
여자55세부터59세생활인구수    0
여자60세부터64세생활인구수    0
여자65세부터69세생활인구수    0
여자70세이상생활인구수       0
dtype: int64

In [15]:
ppl_1903_[ppl_1903_['행정동코드']==11305555][ppl_1903_['기준일ID']==20190304]

<ipython-input-15-4aff372729e3>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppl_1903_[ppl_1903_['행정동코드']==11305555][ppl_1903_['기준일ID']==20190304]


,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
30562,20190304,0,11305555,29501,953,527,657,793,918,843,...,911,883,1024,995,1267,1234,1344,1047,978,3026
31182,20190304,1,11305555,29760,948,524,664,805,957,843,...,925,893,1038,1001,1294,1257,1335,1057,956,3047
31497,20190304,2,11305555,30282,981,543,667,819,1004,886,...,958,907,1072,1027,1314,1259,1343,1076,966,3155
32072,20190304,3,11305555,30331,979,541,658,820,1025,891,...,940,906,1073,1043,1318,1261,1317,1088,975,3140
32421,20190304,4,11305555,30921,1007,557,688,855,1055,937,...,1002,922,1119,1044,1351,1284,1349,1137,965,3086
32793,20190304,5,11305555,30646,1022,565,674,834,1015,953,...,1002,926,1130,1064,1385,1278,1314,1144,952,3072
33104,20190304,6,11305555,29647,1011,559,690,822,984,919,...,1006,917,1069,1019,1317,1254,1280,1109,947,2969
33622,20190304,7,11305555,30096,1021,564,696,784,920,856,...,992,928,1057,1056,1318,1252,1321,1065,968,2883
34253,20190304,8,11305555,29425,894,495,738,708,803,740,...,831,826,970,1053,1351,1224,1347,1024,935,2842
34492,20190304,9,11305555,28446,827,457,719,619,743,720,...,752,748,961,989,1324,1229,1339,966,948,2848


In [23]:
ppl_1904_tmp = ppl_1904_[ppl_1904_['행정동코드']==11305555][ppl_1904_['기준일ID']==20190401].iloc[0]

<ipython-input-23-f763f1d12a40>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppl_1904_tmp = ppl_1904_[ppl_1904_['행정동코드']==11305555][ppl_1904_['기준일ID']==20190401].iloc[0]


## 지도 시각화

In [19]:
import folium
folium.__version__

'0.10.1'

In [39]:
geo_data = 'dataset/map/seoul-dong.geojson'


# 서울시 중심부의 위도, 경도 입니다. 
center = [37.541, 126.986] # 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다. 
m = folium.Map(location=center, zoom_start=10) # Choropleth 레이어를 만들고, 맵 m에 추가합니다. 
folium.Choropleth( geo_data=geo_data, data=ppl_1904_tmp, columns=('행정동코드', '총생활인구수'), key_on='feature.properties.행정동코드', fill_color='BuPu', legend_name='총생활인구수',).add_to(m)
# 맵 m을 출력합니다.
m

UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 173: illegal multibyte sequence